In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
from collections import OrderedDict


class NoDupOrderedDict(OrderedDict):
    def __init__(self, clsname):
        self.clsname = clsname
        super().__init__()

    def __setitem__(self, name, value):
        if name in self:
            raise TypeError(f'{name} already defined in {self.clsname}')
        super().__setitem__(name, value)


class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        d['_order'] = [name for name in clsdict if not name.startswith('_')]
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases):
        return NoDupOrderedDict(clsname)


"""

>>> class A(metaclass=OrderedMeta):
...     def a(): pass
...     def a(): pass
Traceback (most recent call last):
    ...
TypeError: a already defined in A
"""

doctest.testmod()

TestResults(failed=0, attempted=1)

In [4]:
class Typed:
    _expected_type = type(None)

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError(f'{self._name} expects {self._expected_type!s}')

        instance.__dict__[self._name] = value


class Integer(Typed):
    _expected_type = int


class Float(Typed):
    _expected_type = float


class String(Typed):
    _expected_type = str


class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self, name)) for name in self._order)


class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

    def subtotal(self):
        return self.shares * self.price


"""

>>> s = Stock('GOOG', 100, 490.1)
>>> s.name
'GOOG'
>>> s.as_csv()  # doctest: +ELLIPSIS
'GOOG,100,490.1,...'
>>> s._order
['name', 'shares', 'price', 'subtotal']
>>> t = Stock('AAPL', 'a lot', 610.23)
Traceback (most recent call last):
    ...
TypeError: shares expects <class 'int'>
"""

doctest.testmod()

TestResults(failed=0, attempted=5)